<a href="https://colab.research.google.com/github/tylerchi1000/Satander-Customer-Transaction/blob/Test/Satander_Customer_Transaction_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Kaggle Data/santander-customer-transaction-prediction(1)/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Kaggle Data/santander-customer-transaction-prediction(1)/test.csv')


In [ ]:
train.shape

(200000, 202)

In [ ]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [ ]:
train['target'].value_counts()

0    179902
1     20098
Name: target, dtype: int64

In [ ]:
20098/200000

0.10049

In [ ]:
train_features = train.drop(['target', 'ID_code'], axis=1)
train_targets = train['target']
test_features = test.drop(['ID_code'], axis=1)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(train_features, train_targets, test_size = 0.20, random_state = 42)

In [ ]:
from tensorflow import keras
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [ ]:
X_train.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
153248,12.3039,-8.3899,9.1944,8.0649,9.0247,-1.9559,5.1565,21.1631,2.7437,8.5623,...,5.5185,7.9504,0.9184,5.9945,11.0078,-1.0936,-2.3412,8.1712,12.9046,-1.9309
67802,15.4069,2.7820,9.2951,7.1997,8.5359,-4.5422,5.4210,9.9651,4.0623,8.4689,...,3.0063,5.6555,2.1527,1.3518,15.4728,0.2686,6.5523,8.4698,22.0454,1.4756
148889,9.6427,-4.6261,6.9610,5.4054,12.0859,-11.2917,4.5290,13.8605,-0.8366,8.4388,...,3.4351,9.1779,1.5004,1.9895,20.4072,-0.1118,0.5692,9.3290,12.8980,-9.4318
103093,9.6881,-5.6696,11.2709,8.2812,13.9232,-16.1434,4.9664,20.1092,-5.9868,8.4514,...,-4.9494,9.2727,1.1371,3.7435,20.6906,1.3752,7.4442,9.2145,18.2777,-2.5865
104681,7.1128,-2.0830,11.4807,8.3033,10.6180,-6.4743,5.0078,21.0212,-4.9779,9.1260,...,7.3583,8.1992,1.3436,8.8929,21.6711,-2.0557,6.4975,8.3110,13.7728,-5.9028


In [ ]:
y_train.head()


153248    0
67802     0
148889    0
103093    1
104681    0
Name: target, dtype: int64

In [ ]:
#Upsample Minority Class


In [ ]:
df = pd.concat([X_train, y_train], axis=1, join="inner")

In [ ]:
df.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,target
153248,0.596666,0.261741,0.428019,0.612700,0.340608,0.614417,0.467553,0.707785,0.641397,0.639518,...,0.549233,0.387406,0.591312,0.120274,0.437114,0.364832,0.356740,0.325091,0.556163,0
67802,0.752892,0.701230,0.434109,0.547296,0.298458,0.562498,0.511576,0.206579,0.705230,0.626509,...,0.430794,0.488438,0.436890,0.352410,0.579993,0.638217,0.406901,0.793360,0.607476,0
148889,0.462683,0.409804,0.292948,0.411657,0.604582,0.427005,0.363114,0.380931,0.468073,0.622317,...,0.612583,0.435045,0.458101,0.608951,0.540094,0.454298,0.551236,0.324753,0.443175,0
103093,0.464969,0.368754,0.553601,0.629051,0.763017,0.329609,0.435914,0.660614,0.218751,0.624072,...,0.617476,0.405307,0.516441,0.623685,0.696064,0.665634,0.532002,0.600346,0.546288,1
104681,0.335310,0.509847,0.566290,0.630722,0.478002,0.523712,0.442804,0.701434,0.267592,0.718026,...,0.562073,0.422210,0.687715,0.674661,0.336200,0.636532,0.380224,0.369567,0.496334,0


In [ ]:
df.target.value_counts()

0    143999
1     16001
Name: target, dtype: int64

In [ ]:
#Downsample Majority Class (Simple)	
from sklearn.utils import resample
df_majority = df[df.target==0]
df_minority = df[df.target==1]


df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority),     # to match minority class
                                 random_state=42) 
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.target.value_counts()

0    16001
1    16001
Name: target, dtype: int64

In [ ]:
X_train = df_downsampled.drop(['target'], axis = 1)
y_train = df_downsampled['target']

In [ ]:
min_max_scaler = MinMaxScaler()
X_train[X_train.columns] = min_max_scaler.fit_transform(X_train[X_train.columns])

In [ ]:
X_test[X_test.columns] = min_max_scaler.transform(X_test[X_test.columns])

In [ ]:
test_features[train_features.columns] = min_max_scaler.transform(test_features[test_features.columns])

In [ ]:
X_train.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
153248,0.596666,0.261741,0.428019,0.612700,0.340608,0.614417,0.467553,0.707785,0.641397,0.639518,...,0.602806,0.549233,0.387406,0.591312,0.120274,0.437114,0.364832,0.356740,0.325091,0.556163
67802,0.752892,0.701230,0.434109,0.547296,0.298458,0.562498,0.511576,0.206579,0.705230,0.626509,...,0.525588,0.430794,0.488438,0.436890,0.352410,0.579993,0.638217,0.406901,0.793360,0.607476
148889,0.462683,0.409804,0.292948,0.411657,0.604582,0.427005,0.363114,0.380931,0.468073,0.622317,...,0.538768,0.612583,0.435045,0.458101,0.608951,0.540094,0.454298,0.551236,0.324753,0.443175
103093,0.464969,0.368754,0.553601,0.629051,0.763017,0.329609,0.435914,0.660614,0.218751,0.624072,...,0.281055,0.617476,0.405307,0.516441,0.623685,0.696064,0.665634,0.532002,0.600346,0.546288
104681,0.335310,0.509847,0.566290,0.630722,0.478002,0.523712,0.442804,0.701434,0.267592,0.718026,...,0.659355,0.562073,0.422210,0.687715,0.674661,0.336200,0.636532,0.380224,0.369567,0.496334


In [ ]:
#Large Number of Unkown Features. Investigated PCA to see if feature space could
#be reduced.

'''
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_scorerfc = RandomForestClassifier()
rfc.fit(X_train_scaled, y_train)
display(rfc.score(X_train_scaled, y_train))
'''

In [ ]:
#The linear line shows that there isn't a good way to break down the variables
#into principle components. It appears some transformation may have already been
#applied
'''

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCApca_test = PCA(n_components=50)
pca_test.fit(X_train_scaled)sns.set(style='whitegrid')
plt.plot(np.cumsum(pca_test.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.axvline(linewidth=4, color='r', linestyle = '--', x=10, ymin=0, ymax=1)
display(plt.show())evr = pca_test.explained_variance_ratio_
cvr = np.cumsum(pca_test.explained_variance_ratio_)pca_df = pd.DataFrame()
pca_df['Cumulative Variance Ratio'] = cvr
pca_df['Explained Variance Ratio'] = evr
display(pca_df.head(10))
'''

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
forest_cl = RandomForestClassifier()
forest_cl.fit(X_train, y_train)

KeyboardInterrupt: ignored

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(forest_cl, X_train, y_train, cv=3, scoring = 'accuracy')

array([0.74100112, 0.73947689, 0.74660167])

In [ ]:
from sklearn.model_selection import cross_val_predict
forest_cl_y_train_pred = cross_val_predict(forest_cl, X_train, y_train, cv=3)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, forest_cl_y_train_pred)

In [ ]:
from sklearn.model_slection import GridSearchCV

param_grid = {}

In [ ]:
forest_cl2 = RandomForestClassifier()
grid_search = GridSearchCV(forest_cl2, param_grid, cv=5,
                           scoring = 'binary')

Below is a tensorflow model

In [36]:
from keras.wrappers.scikit_learn import KerasClassifier
nn = KerasClassifier()
nn._estimator_type = "classifier"

In [31]:
METRICS = [
      #keras.metrics.TruePositives(name='tp'),
      #keras.metrics.FalsePositives(name='fp'),
      #keras.metrics.TrueNegatives(name='tn'),
      #keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      #keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      #keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [32]:
def create_base_nn(metrics, input_size):

  model = Sequential()
  model.add(Dense(100, activation='relu', input_dim=input_size))
  model.add(Dropout(rate=0.2))
  model.add(Dense(100, activation = 'relu'))
  model.add(Dropout(rate=0.2))
  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics = metrics)
  
  #model.compile(loss = 'binary_crossentropy', 
  #              optimizer = 'adam', 
  #              metrics = ['accuracy'])
  return model

In [33]:
model2 = create_base_nn(metrics = METRICS, input_size=200)

In [ ]:
model2.summary()

In [34]:
result = model2.fit(X_train, y_train, batch_size = 100, epochs = 10, validation_data=(X_test, y_test))

Epoch 1/10
321/321 [==============================] - 3s 6ms/step - loss: 0.6536 - accuracy: 0.6108 - precision: 0.6075 - auc: 0.6602 - val_loss: 0.4007 - val_accuracy: 0.9033 - val_precision: 0.5717 - val_auc: 0.8105
Epoch 2/10
321/321 [==============================] - 2s 5ms/step - loss: 0.5686 - accuracy: 0.7073 - precision: 0.7090 - auc: 0.7794 - val_loss: 0.6661 - val_accuracy: 0.6202 - val_precision: 0.1969 - val_auc: 0.8414
Epoch 3/10
321/321 [==============================] - 1s 4ms/step - loss: 0.5272 - accuracy: 0.7361 - precision: 0.7392 - auc: 0.8142 - val_loss: 0.5519 - val_accuracy: 0.7250 - val_precision: 0.2466 - val_auc: 0.8521
Epoch 4/10
321/321 [==============================] - 1s 4ms/step - loss: 0.5078 - accuracy: 0.7494 - precision: 0.7542 - auc: 0.8298 - val_loss: 0.4754 - val_accuracy: 0.7861 - val_precision: 0.2909 - val_auc: 0.8544
Epoch 5/10
321/321 [==============================] - 1s 5ms/step - loss: 0.4984 - accuracy: 0.7583 - precision: 0.7647 - auc: 0

In [ ]:
from sklearn.metrics import roc_auc_score
y_pred = model2.predict(X_test)
roc_auc_score(y_test, y_pred)

0.8601320867060298

In [43]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators = [('rf', forest_cl), ('lr', log_clf), ('svc',svm_clf)], 
    voting='hard'
)

#voting_clf.fit(X_train, y_train)

In [44]:
for clf in (forest_cl, log_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

RandomForestClassifier 0.74075
LogisticRegression 0.776725
SVC 0.796325
VotingClassifier 0.795175


In [ ]:
sample = pd.read_csv('/content/sample_submission.csv')


In [ ]:
test_features.shape

(200000, 200)

In [ ]:
id_code_test = test['ID_code']
pred = model2.predict(test_features)
pred_ = pred[:,0]
pred_

array([0.23863569, 0.41215286, 0.06348121, ..., 0.00694194, 0.08110669,
       0.21699587], dtype=float32)

In [ ]:
submission = pd.DataFrame({'ID_code': id_code_test, 'target' : pred_})

In [ ]:
from google.colab import files

submission.to_csv('output.csv', encoding = 'utf-8-sig', index=False) 
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sample = pd.read_csv('/content/sample_submission.csv')

In [ ]:
# Some feature Enginering Ideas from Aladdin Persson https://www.youtube.com/watch?v=MOnk75_8b9M&list=PLhhyoLH6IjfxkVb3Yx4mjLdwMWP5S_Hct&index=5
'''As investigated above in PCA there is no corrleation among the features. There appears to have been some transformations
to the data. Feature engineering will hopefully allow the neural network to expose some interesting connections in the data'''



,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0


In [ ]:
#Half of the train data appeared to be copies of other examples.
#Take out unique examples and create a binary column saying if it is unique.
from tqdm import tqdm
col_names = [f"var_{i}" for i in range(200)]
for col in tqdm(col_names):
  count = test[col].value_counts()
  #If count has only 1 then it is unique. Check for all.
  uniques = count.index[count == 1]
  test[col + "_u"] = test[col].isin(uniques)

test["has_unique"] = test[[col + "_u" for col in col_names]].any(axis=1)

In [ ]:
test["has_unique"].sum()

In [ ]:
real_test = test.loc[test["has_unique"], ["ID_code"] + col_names]
fake_test = test.loc[~test["has_unique"], ["ID_code"] + col_names]

#Original Author concatinated train and test set. This exposes the test set

train_and_test = pd.concat([train, real_test], axis=0)


In [ ]:
for col in tqdm(col_names):
  count = train_and_test[col].value_counts().to_dict()
  train_and_test[col + "_unique"] = train_and_test[col].apply(
      lambda x: 1 if count[x] == 1 else 0).values
      #apply each value to those and map to new column
    fake_test[col+"_unique"] = 0
  )

In [ ]:
real_test = train_and_test[train_and_test["ID_code"].str.contains("test")].copy()
real_test.drop(["target"], axis = 1, inplace = True)
engineered_train = train_and_test[train_and_test["ID_code"].str.contains("train")].copy()

In [ ]:
engineered_test = pd.concat([real_test, fake_test], axis = 0)
